# Tensorflow 2.9.1

In [1]:
import tensorflow as tf
tf.version.VERSION

'2.9.1'

# Creating and interaction with "data" directory

In [2]:
import os

DATA_DIR = os.path.join(os.getcwd(), 'data')
MODELS_DIR = os.path.join(DATA_DIR, 'models')
for dir in [DATA_DIR, MODELS_DIR]:
    if not os.path.exists(dir):
        os.mkdir(dir)

# Download and prepare a pretrained object detection model



In [3]:
import tarfile
import urllib.request

# download and extract model
MODEL_DATE = '20200711'
MODEL_NAME = 'ssd_mobilenet_v2_320x320_coco17_tpu-8'
MODEL_TAR_FILENAME = MODEL_NAME + '.tar.gz'
MODELS_DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/'
MODEL_DOWNLOAD_LINK = MODELS_DOWNLOAD_BASE + MODEL_DATE + '/' + MODEL_TAR_FILENAME
PATH_TO_MODEL_TAR = os.path.join(MODELS_DIR, MODEL_TAR_FILENAME)
PATH_TO_CKPT = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'checkpoint/'))
PATH_TO_CFG = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'pipeline.config'))
if not os.path.exists(PATH_TO_CKPT):
    print('downloading model... ', end='')
    urllib.request.urlretrieve(MODEL_DOWNLOAD_LINK, PATH_TO_MODEL_TAR)
    tar_file = tarfile.open(PATH_TO_MODEL_TAR)
    tar_file.extractall(MODELS_DIR)
    tar_file.close()
    os.remove(PATH_TO_MODEL_TAR)
    print('Done')

# labels file
LABEL_FILENAME = 'mscoco_label_map.pbtxt'
LABELS_BASE = \
    'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
PATH_TO_LABELS = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, LABEL_FILENAME))
if not os.path.exists(PATH_TO_LABELS):
    print('downloading label file... ', end='')
    urllib.request.urlretrieve(LABELS_BASE + LABEL_FILENAME, PATH_TO_LABELS)
    print('done')

# Load the SSD_MobileNet model

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

tf.get_logger().setLevel('ERROR')

# enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


# Model Initialization & Inference Setup

In [5]:
import os
from object_detection.utils import config_util
from object_detection.builders import model_builder

configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# load checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

#detection of objects in image
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections, prediction_dict, tf.reshape(shapes, [-1])


# Load label map data (for plotting)
If the model predicts 10, then we know that this shows person.

In [6]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

# Define the video stream

In [7]:
import cv2

cap = cv2.VideoCapture(0)

# Test object detection on a video stream

In [9]:
import numpy as np
import cv2


cap = cv2.VideoCapture('test.mp4') #object_detection/test.mp4

while cap.isOpened():
    ret, image_np = cap.read()
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections, predictions_dict, shapes = detect_fn(input_tensor)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'][0].numpy(),
          (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
          detections['detection_scores'][0].numpy(),
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.40,
          line_thickness=1,
          agnostic_mode=False)

    cv2.imshow('ssd_mobilenet', image_np_with_detections)
    if cv2.waitKey(1) == 13: #Enter Key
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# paths and settings
INPUT_VIDEO  = 'test.mp4'
OUTPUT_VIDEO = 'result.mp4'
THRESHOLD    = 0.45  # Person Threshold
WIDTH, HEIGHT = 1280, 720

# open video
cap = cv2.VideoCapture(INPUT_VIDEO)
fps = cap.get(cv2.CAP_PROP_FPS) or 30

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (WIDTH, HEIGHT))

close_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (WIDTH, HEIGHT))

    inp = tf.convert_to_tensor(np.expand_dims(frame, 0), dtype=tf.float32)
    detections, _, _ = detect_fn(inp)

    # build raw mask of all persons ≥ threshold
    mask = np.zeros((HEIGHT, WIDTH), np.uint8)
    boxes   = detections['detection_boxes'][0].numpy()
    classes = (detections['detection_classes'][0].numpy() + 1).astype(int)
    scores  = detections['detection_scores'][0].numpy()

    for box, cls, score in zip(boxes, classes, scores):
        if cls in category_index and category_index[cls]['name']=='person' and score>=THRESHOLD:
            y1, x1, y2, x2 = (box * np.array([HEIGHT, WIDTH, HEIGHT, WIDTH])).astype(int)
            cv2.rectangle(mask, (x1,y1), (x2,y2), 255, -1)

    # close tiny holes & gaps in mask
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, close_kernel, iterations=1)
    # blur the mask to soften its edges
    mask = cv2.GaussianBlur(mask, (31,31), 0)

    # inpaint using Telea
    cleaned = cv2.inpaint(frame, mask, inpaintRadius=5, flags=cv2.INPAINT_TELEA)

    # sharpening to restore contrast
    sharpen_kernel = np.array([[ 0, -1,  0],
                               [-1,  5, -1],
                               [ 0, -1,  0]])
    cleaned = cv2.filter2D(cleaned, -1, sharpen_kernel)

    # write & show
    out.write(cleaned)
    cv2.imshow('cleaned', cleaned)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
